In [ ]:
import numpy as np
import math

# Leitura do arquivo .txt

In [ ]:
import pandas as pd

In [ ]:
# Abre o arquivo em modo de leitura usando o bloco with
with open("otimo.lp.txt", "r") as arquivo:  #Insira no nome do arquivo no comando da esquerda
    # Lê o conteúdo do arquivo e separa as linhas em uma lista
    linhas = arquivo.readlines()

# Cria uma lista vazia para armazenar as listas de elementos de cada linha
listas = []

# Percorre cada linha
for linha in linhas:
    # Remove espaços em branco no início e no final da linha
    linha = linha.strip()

    # Separa os elementos pelo espaço em branco
    elementos_da_linha = linha.split()

    # Converte os elementos para inteiros ou floats dependendo da presença do ponto
    elementos_convertidos = [float(elemento) if '.' in elemento else int(elemento) for elemento in elementos_da_linha]

    # Adiciona a lista de elementos convertidos à lista de listas
    listas.append(elementos_convertidos)

# Exibe as listas de elementos de cada linha
for lista in listas:
    print(lista)


[1, 2, 2]
[3, 2]
[-1]
[1, 1, 100]
[-1]
[2, 1, 150]
[1, 0]
[1, 0]


# Extração de estruturas importantes do arquivo .txt

In [ ]:
# Função para extrair estruturas para a execução da padronização
def extracao_info(listas):
    obj = listas[0][0]  #objetivo: primeiro elemento da primeira linha do arquivo (0 indica minimização e 1 indica maximização)
    C = listas[1]       #coeficientes da função custo (segunda linha do arquivo)

    if obj:
        C = [-i for i in C]
        obj=0
    n = listas[0][2]    #quantidade de variáveis inicialmente do problema inserido
    #estrutura das restrições
    #res é um dicionário
    res={-1:[],0:[],1:[]}
    # res[-1] armazena os coeficientes das relações de <=
    # res[0] armazena os coeficientes das relações de =
    # res[1] armazena os coeficientes das relações de >=
    # um dos objetivos da padronização é esvaziar as listas res[-1] e res[1], alterando seus antigos valores para que
    # passem a fazer parte de res[0] (ou seja, termos somente restrições de igualdade)
    m = listas[0][1] # quantidade de equações inseridas

    for i in range(m):
        k = 2+2*i       #pega os índices dos elementos que indicam o tipo da restrição
        res[listas[k][0]].append(listas[k+1]) #acrescenta os coeficientes da equação correspondente no grupo correto:
                                                    #res[-1], res[0], ou res[1]


    #estrutura das restrições de sinal
    # sinal também é um dicionário
    # sinal[-1] --> grupo de variáveis <= valor (lista de listas com dois elementos cada:
                    #o primeiro é o indice da variavel var e o segundo é o valor tq  var <= valor)
    # sinal[0] --> grupo de variáveis livres  (lista com os indices das variáveis livres)
    # sinal[1] --> grupo de variáveis >= valor  (lista de listas com dois elementos cada:
                    #o primeiro é o indice da variavel var e o segundo é o valor tq  var >= valor)
    # um dos objetivos da padronização é esvaziar as listas sinal[-1] e sinal[0],
    # além disso, todas as listas de sinal[1] devem ter o segundo elemento igual a 0 para indicar que a variável
    # em questão é >= 0.

    sinal={-1:[],0:[],1:[]}

    var=2*i+4  #indice da primeira restrição de sinal

    cont=0  # representará o índice da variável
    for res_sinal in listas[var:]: # analisa as listas que representam restrições de sinal para popular a lista sinal
        if len(res_sinal)==1:  # signica que se trata de uma variável livre
            sinal[0].append(cont)
        else:
            sinal[res_sinal[0]].append([cont,res_sinal[1]])

        cont=cont+1
    return obj,C,res,n,sinal

In [ ]:
obj,C,res,n,sinal = extracao_info(listas)

In [ ]:
print("obj: ",obj,"\n")
print("C",C,"\n")
print("res: ",res,"\n")
print("n: ",n,"\n")
print("sinal: ",sinal,"\n")

obj:  1 

C [3, 2] 

res:  {-1: [[1, 1, 100], [2, 1, 150]], 0: [], 1: []} 

n:  2 

sinal:  {-1: [], 0: [], 1: [[0, 0], [1, 0]]} 



# Padronização das restrições de igualdade

In [ ]:
def res_pad(res,n,C,sinal):
    cont=0
    if len(res[-1])!=0: #Há pelo menos uma restrição de <=
        total = len(res[-1]) #Total de restrições de <=

        for i in range(total):   #para cada restrição de <=
            sinal[1].append([n+i,0])  #deve-se criar uma nova variável

        n=n+total  # atualiza o total de variáveis

        C.extend([0]*total) #Atualiza C acrescentando valores 0 para cada nova variável
        for elem in range(len(res[0])):  #loop para atualizar a lista res[0]
            aux = [0]*total
            u=res[0][elem][-1]
            res[0][elem]=res[0][elem][:-1]
            res[0][elem].extend(aux)
            res[0][elem].append(u)


        for elem in range(len(res[1])):  #loop para atualizar a lista res[1]
            aux = [0]*total
            u=res[1][elem][-1]
            res[1][elem]=res[1][elem][:-1]
            res[1][elem].extend(aux)
            res[1][elem].append(u)


        for elem in range(total):
            aux = [1 if i == cont else 0 for i in range(total)]
            u=res[-1][elem][-1]
            res[-1][elem]=res[-1][elem][:-1]
            res[-1][elem].extend(aux)
            res[-1][elem].append(u)
            res[0].append(res[-1][elem])


            cont=cont+1
        res[-1]=[]




######################################################################################################
    cont=0



    if len(res[1])!=0:  #Há pelo menos uma restrição de >=
        total = len(res[1])
        for i in range(total):
            sinal[1].append([n+i,0])
        n = n+total
        C.extend([0]*total)
        for elem in range(len(res[0])):
            aux = [0]*total
            u=res[0][elem][-1]
            res[0][elem]=res[0][elem][:-1]
            res[0][elem].extend(aux)
            res[0][elem].append(u)

        for elem in range(total):
            aux = [-1 if i == cont else 0 for i in range(total)]
            u=res[1][elem][-1]
            res[1][elem]=res[1][elem][:-1]
            res[1][elem].extend(aux)
            res[1][elem].append(u)

            res[0].append(res[1][elem])


            cont=cont+1
        res[1]=[]


    total = len(res[0])
    for elem in range(total):
        if res[0][elem][-1]<0:
            res[0][elem] = [-item for item in res[0][elem]]

    A=[]
    b=[]
    util = res[0]
    for elem in range(len(util)):
        A.append(util[elem][:-1])
        b.append(util[elem][-1])

    return A,b,C,res,sinal

In [ ]:
A,b,C,res,sinal=res_pad(res,n,C,sinal)

In [ ]:
A

[[1, 1, 1, 0], [2, 1, 0, 1]]

In [ ]:
b

[100, 150]

In [ ]:
C

[3, 2, 0, 0]

In [ ]:
res

{-1: [], 0: [[1, 1, 1, 0, 100], [2, 1, 0, 1, 150]], 1: []}

In [ ]:
sinal

{-1: [], 0: [], 1: [[0, 0], [1, 0], [2, 0], [3, 0]]}

In [ ]:
#Lida com restrições de <=

for elem in range(len(sinal[-1])):
    s=sinal[-1][elem][0]  #indice
    r=sinal[-1][elem][1]  #valor tq: indice<=valor
    if r!=0: #se o valor for diferente de zero --> criar uma nova restrição de igualdade
        aux=[1 if i == s else 0 for i in range(len(A[0]))]
        print(aux)
        A.append(aux)
        for i in range(len(A)):
            A[i].append(0)
        A[-1][-1]=1
        C.append(0)
        b.append(r)
        sinal[-1][elem][1]=0
    else:  #se o valor for igual a zero --> basta trocar o sinal dos coeficientes
        for h in range(len(A)):
            A[h][s]=-A[h][s]
        sinal[1].append(sinal[-1][elem])
        sinal[-1]=sinal[-1][1:]

        C[s]=-C[s]  #atualiza os coeficientes dos custos
sinal[1].extend(sinal[-1])
sinal[-1]=[]  #Esvazia lista com restrições de sinal <=

In [ ]:
A

[[1, 1, 1, 0], [2, 1, 0, 1]]

In [ ]:
b

[100, 150]

In [ ]:
C

[3, 2, 0, 0]

In [ ]:
sinal

{-1: [], 0: [], 1: [[0, 0], [1, 0], [2, 0], [3, 0]]}

In [ ]:
# Analisa o grupo de variáveis livres
for elem in range(len(sinal[0])):

    y=sinal[0][elem]          #convetendo os elemento de sinal[0] para satisfazerem a restrição de <=
    sinal[1].append([y,0])    #e passarem para o grupo sinal[1]

    for i in range(len(A)): # exemplo, a variável r será substituída por b-c, com b e c >=0

        a=A[i][y]         #pega o coeficiente que multiplicava r
        A[i].append(-a)    #acrescenta uma nova variável negativa
                            #o indice de r é reutilizado para representar a nova variável b

    if C[y]!=0:
        u=C[y]
        C.append(-u)



sinal[0]=[]  #Esvazia lista de variáveis livres


In [ ]:
A

[[1, 1, 1, 0], [2, 1, 0, 1]]

In [ ]:
b

[100, 150]

In [ ]:
C

[3, 2, 0, 0]

In [ ]:
sinal

{-1: [], 0: [], 1: [[0, 0], [1, 0], [2, 0], [3, 0]]}

In [ ]:
# Analisa o grupo de variáveis >= a um valor diferente de zero

for elem in range(len(sinal[1])):
    s=sinal[1][elem][0] #indice
    r=sinal[1][elem][1] #valor tq: indice<=valor
    sinal[1][elem][1]=0  # troca o valor para zero
    if r!=0:
        aux=[1 if i == s else 0 for i in range(len(A[0]))]
        A.append(aux)  #cria uma nova restrição
        for i in range(len(A)):
            A[i].append(0)
        A[-1][-1]=-1
        res[0].append(A[-1])  #-------------#
        res[0][-1].append(r)  #-------------#
        C.append(0)  #atualiza os coeficientes do custo
        b.append(r)  #atualiza b, acrescenta um novo elemento r



In [ ]:
A

[[1, 1, 1, 0], [2, 1, 0, 1]]

In [ ]:
b

[100, 150]

In [ ]:
C

[3, 2, 0, 0]

In [ ]:
res

{-1: [], 0: [[1, 1, 1, 0, 100], [2, 1, 0, 1, 150]], 1: []}

In [ ]:
sinal

{-1: [], 0: [], 1: [[0, 0], [1, 0], [2, 0], [3, 0]]}

# Mostra o problema na forma canônica após as modificações

In [ ]:
def mostrar_lp_can(C,res,sinal):
    ######Custo#####
    custo = ""
    for i in range(len(C)):
        custo = custo+str(C[i])+"*"+"x"+str(i)+" + "
    custo = custo[:-3]

    ######Restrições#####
    restricoes = []
    for rst in range(len(res[0])):
        r = ""
        for elem in range(len(res[0][0])-1):

            r = r+"("+str(res[0][rst][elem])+")"+"*"+"x"+str(elem)+" + "


        r = r[:-2]+" = "+str(res[0][rst][-1])+r[-1]

        restricoes.append(r)

    #####Sinais#####
    r_sinais = []
    for s in range(len(sinal[1])):
        r = "x"+str(sorted(sinal[1])[s][0])+" >= "+str(0)
        r_sinais.append(r)
    print("#####----- PRIMAL -----#####\n")
    print("min ",custo)
    print("s.a.")
    for z in restricoes:
        print("\t"+z)

    for z in r_sinais:
        print("\t"+z)

In [ ]:
mostrar_lp_can(C,res,sinal) # botar em ordem as restrições de sinal

#####----- PRIMAL -----#####

min  3*x0 + 2*x1 + 0*x2 + 0*x3
s.a.
	(1)*x0 + (1)*x1 + (1)*x2 + (0)*x3  = 100 
	(2)*x0 + (1)*x1 + (0)*x2 + (1)*x3  = 150 
	x0 >= 0
	x1 >= 0
	x2 >= 0
	x3 >= 0


In [ ]:
Aprimal = np.array(A)

In [ ]:
Aprimal

array([[1, 1, 1, 0],
       [2, 1, 0, 1]])

In [ ]:
bprimal = np.array(b).T

In [ ]:
bprimal

array([100, 150])

In [ ]:
Cprimal = np.array([C])

In [ ]:
C

array([[3, 2, 0, 0]])

#Fase 1

##Colocar a forma padrão na forma inicial da fase 1

In [ ]:
def normalizePhase1(M, b, C): #Função que "normaliza" os dados do LP original para a forma do LP fase 1
  m = M.shape[0] #Calcula o numero de linhas da matriz M que é composta pelos coeficientes das restrições
  n = M.shape[1] + 1 #Calcula o numero de colunas + 1 da matriz M (número de variáveis após a padronização + 1)

  y_matrix = np.eye(m) #cria uma matriz identidade de ordem m

  A = np.hstack((M, y_matrix)) #Matriz de coeficientes composta pelos coeficientes originais adicionado dos coeficientes das variáveis artificiais y

  c_1 = np.hstack((np.zeros((1, n - 1)), np.ones((1, m)))) #Criando um vetor de coeficientes que representa o Lp fase 1, ou seja, as variáveis que não são auxiliares tem coeficiente 0 e as artificais 1

  restrictions = []
  cost = ''
  signals = ''


  for k in range(0, n - 1):
      cost = cost + "("+str(c_1[0][k])+")*x" + str(k + 1) + " + "
      signals = signals + "x"+str(k + 1)+", "
  for l in range(n - 1, n + m - 2):
    cost = cost + '('+str(c_1[0][l])+")*y" + str(l - n + 2) + " + "
    signals = signals + "y"+str(l - n + 2)+", "
  cost = cost + '('+str(c_1[0][n + m - 2])+")*y" + str(l - n + 3)
  signals = signals + "y"+str(l - n + 3)+" >= 0"

  for j in range(0, m):
    restriction = ''
    signal = ''
    for k in range(0, n - 1):
      restriction = restriction + '('+str(A[j][k])+")*x" + str(k + 1) + " + "
    for l in range(n - 1, n + m - 2):
      restriction = restriction + '('+str(A[j][l])+")*y" + str(l - n + 2) + " + "
    restriction = restriction + '('+str(A[j][n + m - 2])+")*y" + str(l - n + 3)
    restriction = restriction + " = " + str(b[j]) +"\n"
    restrictions.append(restriction)

  print("\nForma padrão do simplex fase 1:")
  print(f"min {cost}\ns.a:\n")
  for my_list in restrictions:
    message = " ".join(my_list)
    print(message)
  print(signals)

  Phase1(A = A, b = b, c = c_1, M = M, C = C, m = m, n = n) # Chama a função que aplica fase 1 passando a matriz A que é a modificação de M, o vetor b inalterado, o vetor de coeficientes "artificiais", M, o vetor C original e as dimensões de M


In [ ]:
def Phase1(A, b, c, M, C, j = None, k = None, m = None, n = None, B = None, N = None, C_B = None, C_N = None, index_B = None, index_N = None, cont = None): #Função de aplicação da Fase 1
  columns = A.shape[1] #Obtém o número de colunas de A
  if(B is None): #Verifica se B ainda não foi definida, o que ocorre na primeira chamada da função, pois a base ainda será calculada como sendo a matriz identidade em A
    cont = 1 #Identificar a n-ésima solução encontrada
    B = A[ : , n - 1:  ].copy() # A criação da primeira bfs, a matriz identidade em A
    N = A[ : , : n - 1].copy() # a matriz restante é a N
    C_B = c[ : , n - 1:  ].copy() #Os coeficientes das variáveis básicas
    C_N = c[ : , : n - 1].copy() #Coeficientes das variáveis não básicas
    index_B = np.arange(n - 1, columns) #Índices para ajudar a saber quem atualmente está na base, guardando o valor de sua coluna em A
    index_N = np.arange(0, n - 1) #Índices para ajudar a saber quem atualmente não está na base, guardando o valor de sua coluna em A
  else: #Se a matriz B já foi criada, fazer a etapa de realocar quem entra e quem sai da mesma
    B[: , [k]], N[: , [j]] = N[: , [j]], B[: , [k]] #Troca a coluna j de N com a coluna k de B
    C_B[: , [k]], C_N[: , [j]] = C_N[: , [j]], C_B[: , [k]] #Atualiza os coeficientes de custo de quem está e não está na base, usando o mesmo raciocínio
    index_B[k], index_N[j] = index_N[j], index_B[k] #O mesmo para atualizar os índices das colunas de quem está e não está na base relativos a matriz A

  print(f'{cont}ª solução encontrada na fase 1, índices das variáveis básicas: {index_B}\n')
  cont += 1


  B_1 = np.linalg.inv(B) #Calcula a inversa de B

  xb = np.dot(B_1, b) #Calcula xb multiplicando a inversa de B com o vetor b

  test = C_N - np.linalg.multi_dot([C_B, B_1, N]) #Cálculo para o teste de otimalidade

  if np.all(test >= 0): #Se test >= 0, o valor ótimo foi atingido, passou no teste
    #Verificar se vai pra fase 2 ou auxiliar:
    #se algum y esta na base, e é diferente de zero, o problema é inviável;
    if np.any(index_B >= n - 1): #verificando se tem y na base a partir da verificação dos valores dos índices relativos a matriz A (considerando que o problema tem n - 2 variaveis, os valores dos indices das variaveis artificiais são maiores ou iguais a n - 1)
      for i in np.where(index_B >= n - 1)[0]: #iterando sobre os indices de y que estão na base
        if(xb[i] != 0): #verificando se o valor do y é maior que 0
          print("\nProblema inviável") #se algum y está na base e é diferente de 0, o problema é inviável
          return
      return normalizeAuxiliar(A = A, b = b, C = C, m = m, n = n, index_B = index_B, index_N = index_N) #se está na base, mas tem valor 0, vai pra o auxiliar, passando como principais parametros a matriz A modificada, b, C original, e os índices de quem está na base, para ser possível formular a bfs do LP auxiliar
    #Se nenhum y está na base, vai para a fase 2
    else:
      return normalizeAuxiliar(A = A, b = b, C = C, m = m, n = n, index_B = index_B, index_N = index_N)
      #return Phase2(A = M, b = b, c = C, n = n, index_B = index_B) #Uma bfs foi encontrada e é bfs para a fase 2, chama a função de aplicação da fase 2 passando como principais parametros a matriz M original, b, C original e os índices de quem está na base para ser possível formular a bfs inicial do LP original
  #Verifica os valores negativos do custo, pega os indices
  #coloca o indice de cada um em index_N e ve quem é o menor. então j = indice
  index_C_N = np.where(test < 0)

  index_j = index_C_N[1]
  j = index_j[np.argmin(index_N[index_j])] #Usando regra de Bland para calcular j
  db = -np.dot(B_1, N[:, j : j + 1]) #Cálculo de db sendo -B^(-1).N_j
  t = []
  for xb_i, db_i in zip(xb, db): #itera sobre os elementos de xb e db
    if db_i < 0: #Para db_i < 0, calcula -xb_i/db_i
      t.append(- xb_i/db_i) #Adiciona o resultado em t
    else:
      t.append(math.inf)
  min_t = min(t)  #Escolha de t = min{-xb_i/db_i, tal que db_i < 0}
  draws = t.count(min_t) #Verifica se há empate no valor mínimo
  if draws > 1: #Se há mais que um valor mínimo para escolha de t
    index_C_B = np.where(t == min_t) #Pega os índices onde ocorrem o valor mínimo na lista t
    index_k = index_C_B[0] #Simplifica o valor de index_C_B
    k = index_k[np.argmin(index_B[index_k])] #Seleção do k usando a regra de Bland
  else: #se não ocorrem empates
    k = t.index(min_t) #k recebe i tal que -xb_i/db_i (db_i < 0) é mínimo


  return Phase1(A = A, b = b, c = c, M = M, C = C, j = j, k = k, m = m, n = n, B = B, N = N, C_B = C_B, C_N = C_N, index_B = index_B, index_N = index_N, cont = cont) #Faz a chamada recursiva da Fase 1 para uma nova busca por uma bfs


#Auxiliar

In [ ]:
def normalizeAuxiliar(A, b, C, m =None, n = None, index_B = None, index_N = None): #Função para passar as variáveis para a forma padrão de aplicação do LP auxiliar
  half_1 = np.zeros((n - 1)) #Vetor de zeros com tamanho n - 1
  half_2 = np.ones((m + 1)) #Vetor de uns com tamanho m + 1
  integral = np.hstack((half_1, half_2)) #Junção dos dois vetores
  vertical = np.zeros((m, 1)) #vetor coluna de zeros com m elementos
  A = np.hstack((A, vertical)) #primeira modificação em A
  A = np.vstack((A, integral)) #última modificação em A, deixando-a na forma correta para aplicar o LP auxiliar, ou seja, considerou-se o acréscimo da variável z
  C_yz = np.zeros((1, m + 1)) #

  C = np.hstack((C, C_yz)) #Modifica o vetor de custos considerando a nova variável z
  b = np.hstack((b, np.array([0]))) #adiciona 0 ao vetor b, considerando a variável z acrescida


  restrictions = []
  cost = ''
  signals = ''

  for k in range(0, n - 1):
      cost = cost + "("+str(C[0][k])+")*x" + str(k + 1) + " + "
      signals = signals + "x"+str(k + 1)+", "
  for l in range(n - 1, n + m - 1):
    cost = cost + '('+str(C[0][l])+")*y" + str(l - n + 2) + " + "
    signals = signals + "y"+str(l - n + 2)+", "
  cost = cost + '('+str(C[0][n + m - 1])+")*z"
  signals = signals + "z >= 0"

  for j in range(0, m):
    restriction = ''
    for k in range(0, n - 1):
      restriction = restriction + '('+str(A[j][k])+")*x" + str(k + 1) + " + "
    for l in range(n - 1, n + m - 1):
      restriction = restriction + '('+str(A[j][l])+")*y" + str(l - n + 2) + " + "
    restriction = restriction + '('+str(A[j][n + m - 2])+")*z"
    restriction = restriction + " = " + str(b[j]) +"\n"
    restrictions.append(restriction)

  print("\nForma padrão do LP auxiliar:")
  print(f"min {cost}\ns.a:\n")
  for my_list in restrictions:
    message = " ".join(my_list)
    print(message)
  print(signals)

  LpAuxiliar(A = A, b = b, c = C, n = n, index_B = index_B, index_N = index_N) #Chamada da função do LP auxiliar


In [ ]:
def LpAuxiliar(A, b, c, j = None, k = None, n = None, B = None, N = None, C_B = None, C_N = None, index_B = None, index_N = None, cont = None): #Função de aplicação do LP auxiliar
  columns = A.shape[1]
  if(B is None):#Verifica se B ainda não foi definida, o que ocorre na primeira chamada da função, pois a base ainda será calculada como sendo a bfs encontrada pela fase 1
    cont = 1 #Identificar a n-ésima solução encontrada
    index_B = np.hstack((index_B, np.array(columns - 1))) #Atualiza a lista de índices para considerar o índice da variável z que inicia na base
    B = A[ : , index_B].copy() #Inicializa B como sendo a bfs encontrada pela fase 1 utilizando os indices das colunas de A que devem compor B
    N = A[ : , index_N].copy() #Criação da matriz N inicial como sendo as colunas de A que não estão em B
    C_B = c[ : , index_B].copy() #custos associados as variáveis básicas
    C_N = c[ : , index_N].copy() #custos associados as variáveis não básicas
  else: #Se B já foi definida
    B[: , [k]], N[: , [j]] = N[: , [j]], B[: , [k]] #Faz o pivoteamento entre B e N com relação as colunas k e j
    C_B[: , [k]], C_N[: , [j]] = C_N[: , [j]], C_B[: , [k]] #Atualiza os custos associados as variáveis básicas e não básicas
    index_B[k], index_N[j] = index_N[j], index_B[k] #Atualiza os índices em A das variáveis básicas e não básicas

  print(f'{cont}ª solução encontrada no LP auxiliar, índices das variáveis básicas: {index_B}\n')
  cont += 1

  B_1 = np.linalg.inv(B) #Calcula a inversa de B

  xb = np.dot(B_1, b) #Calcula xb multiplicando a inversa de B com o vetor b

  test = C_N - np.linalg.multi_dot([C_B, B_1, N]) #Cálculo para o teste de otimalidade

  if np.all(test >= 0): #Se test >= 0, o valor ótimo foi atingido, passou no teste
    x = np.zeros((index_B.shape[0] + index_N.shape[0])) #Vetor para armazenar os valores das variáveis todas
    for x_index, x_value in zip(index_B, xb): #itera sobre os valores dos indices das variáveis básicas e os valores em si das variáveis básicas
      #print(f'index: {x_index}, value: {x_value}')
      x[x_index] = x_value #popula o vetor x, de modo que as respectivas variaveis do problema recebam seus respectivos valores
    f = np.dot(c, np.transpose(x))[0] #f é o valor da função objetiva, calculada pelo produto entre c e x que é o mesmo valor mínimo para o LP original (se for de minimizar)
    print(f'\nValor mínimo encontrado: {f}\n x = {x}')
    return

  #Verifica os valores negativos do custo, pega os indices
  #coloca o indice de cada um em index_N e ve quem é o menor. então j = indice
  index_C_N = np.where(test < 0)
  index_j = index_C_N[1]
  j = index_j[np.argmin(index_N[index_j])] #Usando regra de Bland para calcular j
  db = -np.dot(B_1, N[:, j : j + 1]) #Cálculo de db sendo -B^(-1).N_j
  if np.all(db >= 0): #se db >= 0
    print("\nProblema ilimitado") #O problema é ilimitado
    return


  t = []
  for xb_i, db_i in zip(xb, db): #itera sobre os elementos de xb e db
    if db_i < 0: #Para db_i < 0, calcula -xb_i/db_i
      t.append(- xb_i/db_i) #Adiciona o resultado em t
    else:
      t.append(math.inf)

  min_t = min(t) #Escolha de t = min{-xb_i/db_i, tal que db_i < 0}
  draws = t.count(min_t) #Verifica se há empate no valor mínimo
  if draws > 1: #Se há mais que um valor mínimo para escolha de t
    index_C_B = np.where(t == min_t) #Pega os índices onde ocorrem o valor mínimo na lista t
    index_k = index_C_B[0] #Simplifica o valor de index_C_B
    k = index_k[np.argmin(index_B[index_k])] #Seleção do k usando a regra de Bland
  else: #se não ocorrem empates
    k = t.index(min_t) #k recebe i tal que -xb_i/db_i (db_i < 0) é mínimo

  return LpAuxiliar(A = A, b = b, c = c, j = j, k = k, n = n, B = B, N = N, C_B = C_B, C_N = C_N, index_B = index_B, index_N = index_N, cont = cont) #Faz a chamada recursiva do LP auxiliar para uma nova busca por uma solução ótima, que será ótima também para o LP original


#Fase 2

In [ ]:
def Phase2(A, b, c, j = None, k = None, n = None, B = None, N = None, C_B = None, C_N = None, index_B = None, index_N = None, cont = None): #Função para aplicação da fase 2
  if(B is None):#Verifica se B ainda não foi definida, o que ocorre na primeira chamada da função, pois a base ainda será calculada como sendo a bfs encontrada pela fase 1
    cont = 1 #Identificar a n-ésima solução encontrada
    B = A[ : , index_B].copy() #Define a primeira bfs com base nas informações repassadas pela fase 1
    N = np.delete(A.copy(), index_B, axis=1) #A matriz N é formada pelas colunas de A que não estão em B
    C_B = c[ : , index_B].copy() #custos associados as variáveis básicas
    C_N = np.delete(c.copy(), index_B, axis=1) #custos associados as variáveis não básicas
    index_N = np.delete(np.arange(0, A.shape[1]), index_B) #inicializa o vetor que guarda os índices das variáveis não básicas
  else: #Se B já foi definida
    B[: , [k]], N[: , [j]] = N[: , [j]], B[: , [k]] #Faz o pivoteamento entre B e N com relação as colunas k e j
    C_B[: , [k]], C_N[: , [j]] = C_N[: , [j]], C_B[: , [k]] #Atualiza os custos associados as variáveis básicas e não básicas
    index_B[k], index_N[j] = index_N[j], index_B[k] #Atualiza os índices em A das variáveis básicas e não básicas

  print(f'{cont}ª solução encontrada na Fase 2, índices das variáveis básicas: {index_B}\n')
  cont += 1

  B_1 = np.linalg.inv(B) #Calcula a inversa de B

  xb = np.dot(B_1, b) #Calcula xb multiplicando a inversa de B com o vetor b (Valores das variáveis básicas)


  test = C_N - np.linalg.multi_dot([C_B, B_1, N]) #Cálculo para o teste de otimalidade


  #Verificar se passou no teste de otimalidade:
  if np.all(test >= 0): #Se test >= 0, o valor ótimo foi atingido, passou no teste
    x = np.zeros((index_B.shape[0] + index_N.shape[0])) #Vetor para armazenar os valores das variáveis todas

    for x_index, x_value in zip(index_B, xb): #itera sobre os valores dos indices das variáveis básicas e os valores em si das variáveis básicas
      #print(f'index: {x_index}, value: {x_value}')
      x[x_index] = x_value #popula o vetor x, de modo que as respectivas variaveis do problema recebam seus respectivos valores
    f = np.dot(c, np.transpose(x))[0] #f é o valor da função objetiva, calculada pelo produto entre c e x
    print(f'\nValor mínimo encontrado: {f}\n x = {x}')
    return

  #Verifica os valores negativos do custo, pega os indices
  #coloca o indice de cada um em index_N e ve quem é o menor. então j = indice
  index_C_N = np.where(test < 0)
  index_j = index_C_N[1]
  j = index_j[np.argmin(index_N[index_j])] #Usando regra de Bland para calcular j
  db = -np.dot(B_1, N[:, j : j + 1]) #Cálculo de db sendo -B^(-1).N_j
  print(f'db = {db}')
  if np.all(db >= 0): #se db >= 0
    print("\nProblema ilimitado") #O problema é ilimitado
    return

  t = []
  for xb_i, db_i in zip(xb, db): #itera sobre os elementos de xb e db
    if db_i < 0: #Para db_i < 0, calcula -xb_i/db_i
      t.append(- xb_i/db_i) #Adiciona o resultado em t
    else:
      t.append(math.inf)

  min_t = min(t) #Escolha de t = min{-xb_i/db_i, tal que db_i < 0}
  draws = t.count(min_t) #Verifica se há empate no valor mínimo
  if draws > 1: #Se há mais que um valor mínimo para escolha de t
    index_C_B = np.where(t == min_t) #Pega os índices onde ocorrem o valor mínimo na lista t
    index_k = index_C_B[0] #Simplifica o valor de index_C_B
    k = index_k[np.argmin(index_B[index_k])] #Seleção do k usando a regra de Bland
  else: #se não ocorrem empates
    k = t.index(min_t) #k recebe i tal que -xb_i/db_i (db_i < 0) é mínimo

  return Phase2(A = A, b = b, c = c, j = j, k = k, n = n, B = B, N = N, C_B = C_B, C_N = C_N, index_B = index_B, index_N = index_N, cont = cont) #Faz a chamada recursiva da fase 2 para uma nova busca por uma solução ótima


#Testes

Chame a função normalizePhase1() passando como parâmetros Aprimal, bprimal e Cprimal, que são arrays numpy, ela retorna o valor obtido na aplicação do algoritmo primal simplex

In [ ]:
resultado = normalizePhase1(Aprimal, bprimal, Cprimal)

Forma padrão do simplex fase 1:
min (0.0)*x1 + (0.0)*x2 + (0.0)*x3 + (0.0)*x4 + (1.0)*y1 + (1.0)*y2
s.a:

( 1 . 0 ) * x 1   +   ( 1 . 0 ) * x 2   +   ( 1 . 0 ) * x 3   +   ( 0 . 0 ) * x 4   +   ( 1 . 0 ) * y 1   +   ( 0 . 0 ) * y 2   =   1 0 0 

( 2 . 0 ) * x 1   +   ( 1 . 0 ) * x 2   +   ( 0 . 0 ) * x 3   +   ( 1 . 0 ) * x 4   +   ( 0 . 0 ) * y 1   +   ( 1 . 0 ) * y 2   =   1 5 0 

x1, x2, x3, x4, y1, y2 >= 0
1ª solução encontrada na fase 1, índices das variáveis básicas: [4 5]
2ª solução encontrada na fase 1, índices das variáveis básicas: [4 0]
3ª solução encontrada na fase 1, índices das variáveis básicas: [1 0]
Forma padrão do LP auxiliar:
min (3.0)*x1 + (2.0)*x2 + (0.0)*x3 + (0.0)*x4 + (0.0)*y1 + (0.0)*y2 + (0.0)*z
s.a:

( 1 . 0 ) * x 1   +   ( 1 . 0 ) * x 2   +   ( 1 . 0 ) * x 3   +   ( 0 . 0 ) * x 4   +   ( 1 . 0 ) * y 1   +   ( 0 . 0 ) * y 2   +   ( 0 . 0 ) * z   =   1 0 0 

( 2 . 0 ) * x 1   +   ( 1 . 0 ) * x 2   +   ( 0 . 0 ) * x 3   +   ( 1 . 0 ) * x 4   +   ( 0 . 0 ) * y 1

# Dual Simplex

In [ ]:
def mostra_dual(b,A,C):
    aux = C
    C = b
    b = aux
    array = np.array(A, dtype=object)
    n = array.shape[1]
    grupo = [i for i in range(len(C))] # lista com os indices das variáveis livres

    A= array.T

    #####Custo#####

    custo = ""
    for i in range(len(C)):
        custo = custo+"("+str(C[i])+")"+"*"+"u"+str(i)+" + "
    custo = custo[:-3]
    print("#####----- DUAL -----#####\n")
    print("max ",custo)
    print("s.a.")



    #####Restrições#####
    restricoes = []

    for rst in range(A.shape[0]):

        if sum(1 for elemento in A[rst] if elemento != 0) == 1:  #verifica se há um único coeficiente diferente de zero,
                                                            #pois assim a inequação vai servir como uma restrição de sinal
            for t in range(len(A[rst])):
                if A[rst][t]!=0:
                    temp = t

                    if t in grupo:
                        grupo.remove(t)         #dessa forma, posso excluir a variável do grupo de livres

        r = ""
        for elem in range(A.shape[1]):

            r = r+"("+str(A[rst][elem])+")"+"*"+"u"+str(elem)+" + "


        r = r[:-2] + "<=" + str(b[rst])

        restricoes.append(r)

    for z in restricoes:
        print("\t"+z)

    livres = ""

    for i in grupo:
        livres = livres + "u"+str(i)+", "


    livres = livres[:-2] + " livres"


    if len(grupo)!=0:
        print("\n\t", livres)

In [ ]:
mostra_dual(b,A,C)

#####----- DUAL -----#####

max  (100)*u0 + (150)*u1
s.a.
	(1)*u0 + (2)*u1 <=3
	(1)*u0 + (1)*u1 <=2
	(1)*u0 + (0)*u1 <=0
	(0)*u0 + (1)*u1 <=0


# Teste da dbfs inserida pelo usuário

In [ ]:
def Teste_dbfs(T,C,candidatos):
    T = pd.DataFrame(T)
    C = pd.DataFrame(C)
    n = T.shape[1]
    indices = [i for i in range(n)]
    indices = pd.DataFrame(indices)
    indices = indices.T

    indB = indices.loc[:,candidatos]
    B=T.loc[:, candidatos]

    # Verificar se a matriz é invertível
    determinante = np.linalg.det(B)

    if determinante == 0:
        print("B não é invertível!")
        return False
    else:
#         print("B é invertível.")
        B_inversa = np.linalg.inv(B)

    N = T[T.columns.difference(candidatos)]
    indN = indices[indices.columns.difference(candidatos)]
    CB = C.loc[candidatos,:]
    CN = C.T[C.T.columns.difference(candidatos)]

    u = np.matmul(CB.T,B_inversa)
    uT = np.matmul(u,T)
    cond = np.all(uT<= C.T)
    if cond:

        return B,N,CB.T,CN,u,indB,indN
    else:
        print("Condição uA <= C Falhou!")
        return False


# Teste de Otimalidade

In [ ]:
def Teste_Otim(B,b,u):


    B_inversa = np.linalg.inv(B)
    B_inv_b = np.matmul(B_inversa,b)
    y=[]
    passou = True
    for i in range(len(B_inv_b)):
        if B_inv_b[i]<0:
            y.append(i)
            passou = False

    if passou:

        return True

    return y #caso o teste falhe, retornar uma lista dos indices dos valores negativos de B_inv_b


# Encontrar indice k da variável que vai sair na base por meio da Regra de Bland

In [ ]:
def find_k(indices,y):

    p=[indices[u] for u in y]

    menor_elemento = np.min(p)  #econtra o menor indice
    k = np.where(indices == menor_elemento)[0]
    return k

# Identificar a direção de movimento

In [ ]:
def direcao(B_inversa,k):
    d = B_inversa[k]
    d=np.array(d)
    d = -d
    return d

# Obter o índice da variável que vai entrar da base por meio do cálculo da quantidade de movimento

In [ ]:
def qtd_mov(d,N,CN,u):
    dN = np.matmul(d,N)
    uN = np.matmul(u,N)
    num =CN-uN
    y=[]

    for i in range(len(dN)):
        if dN[i]>0:
            y.append(i)

    lista = []
    for i in range(len(y)):

        lista.append(num[y[i]]/dN[y[i]])
    t = np.min(lista)   #menor elemento
    k = np.where(lista == t)[0][0] #indice do menor elemento
    j = y[k]
    return j

# Pivoteamento: efetuar mudanças nas listas de variáveis básicas e não básicas

In [ ]:
def Pivoteamento(B,k,N,j,indB,indN):

    # Índices das colunas a serem trocadas
    # k Índice da coluna em B
    # j Índice da coluna em N

    # Realizar a troca

    aux = N.iloc[:,j].copy()

    temp = B.iloc[:,k].copy()
    N.iloc[:,j] = temp
    B.iloc[:,k] = aux

    indN = pd.DataFrame(np.array(indN).T)
    indB = pd.DataFrame(np.array(indB).T)

    aux = indN.iloc[:,j].copy()
    temp = indB.iloc[:,k].copy()
    indN.iloc[:,j] = temp
    indB.iloc[:,k] = aux



    return B,N,indB,indN

# Algoritmo DualSimplex

In [ ]:
def DualSimplex(T,C,b,candidatos):

    B,N,CB,CN,u,indB,indN = Teste_dbfs(T,C,candidatos)
    indices = [i for i in range(T.shape[1])]   #armazena os indices das variáveis. Será últil para a operação de pivoteamento
    CB = list(CB.iloc[0])
    CN = list(CN.iloc[0])
    while True:

        B_inversa = np.linalg.inv(B)
        if not isinstance(Teste_Otim(B,b,u), bool):

            y = Teste_Otim(B,b,u)
            k = find_k(indices,y)[0]  #encontra a variável que entra na base
            d = direcao(B_inversa,k) # encontra a direção de movimento
            dN = np.matmul(d,N)

            aux = np.all(dN<=0)     # verifica se algum elemento de dN é negativo
            if aux:
                print("O DUAL É ILIMITADO") #caso exista, o problema é ilimitado
                return False

            j = qtd_mov(d,N,CN,u)  #o problema não é ilimitado    ||||| Encontrar o indice da variável que vai entrar na base

            indN = indN.T
            indB = indB.T


            B,N,indB,indN = Pivoteamento(B,k,N,j,indB,indN)



            temp = CB[k]
            temp2 = CN[j]
            CB[k]=temp2   #atualiza CB

            CN[j] = temp   #atualiza CN



        else:
            print("#####----PASSOU NO TESTE DE OTIMALIDADE----#####\n")
            B = np.array(B)
            CB = np.array(CB).T

            B_inversa = np.linalg.inv(B)
            u = np.matmul(CB,B_inversa)
            x = np.matmul(B_inversa,b)
            return u,indB,indN,x,CB,CN,b




# Inserir candidatos a dbfs

In [ ]:
# Ao rodar esta célula será solicitado que o usuário insira índices candidatos
candidatos = []
qtd = len(A)
for i in range(qtd):
    aux = input("Digite um índice de 0 até "+ str(len(A[0])-1)+"\n")
    candidatos.append(int(aux))

Digite um índice de 0 até 3
2
Digite um índice de 0 até 3
3


# Testa a candidata a dbfs e, caso o resultado seja positivo, aplica o Dual Simplex

In [ ]:
A = np.array(A)
if Teste_dbfs(A,C,candidatos): #Verifica se os índices em candidatos formam uma dbfs
    u,indB,indN,x,CB,CN,b = DualSimplex(A,C,b,candidatos)

    if u.any == False:
        print("O DUAL É ILIMITADO")

    else:
        print("u: ",u)
        print("Custo ótimo: ", np.matmul(u,b))
else:
    print("NÃO É UMA DBFS")


#####----PASSOU NO TESTE DE OTIMALIDADE----#####

u:  [0. 0.]
Custo ótimo:  0.0
